In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.datasets import cifar10

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

Define Transformations

In [6]:
transform=transforms.Compose([
    transforms.ToTensor()
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Load Dataset

In [7]:
train_dataset=datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset=datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:03<00:00, 47.9MB/s]


Create DataLoaders

In [8]:
train_loader=DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=64, shuffle=False)

In [11]:
print(f'Training dataset size: {len(train_dataset)}')
print(f'Testing dataset size: {len(test_dataset)}')

Training dataset size: 50000
Testing dataset size: 10000


Define a CNN Architecture

In [22]:
# Define a CNN by subclassing nn.Module
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # First convolutional layer:
        # - Input channels: 3 (RGB image)
        # - Output channels: 6 feature maps
        # - Kernel size: 5x5
        self.conv1 = nn.Conv2d(3, 6, 5)

        # Max pooling layer:
        # - Kernel size: 2x2
        # - Stride: 2
        self.pool = nn.MaxPool2d(2, 2)

        # Second convolutional layer:
        # - Input: 6 channels from conv1
        # - Output: 16 channels
        # - Kernel size: 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)

        # First fully connected (dense) layer:
        # - Input: flattened tensor of size 16*5*5 (after conv and pooling)
        # - Output: 120 neurons
        self.fc1 = nn.Linear(16 * 5 * 5, 120)

        # Second fully connected layer:
        # - Input: 120 neurons
        # - Output: 84 neurons
        self.fc2 = nn.Linear(120, 84)

        # Final output layer:
        # - Input: 84 neurons
        # - Output: 10 classes (e.g., CIFAR-10 classification)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Apply first conv layer + ReLU + MaxPool
        x = self.pool(F.relu(self.conv1(x)))

        # Apply second conv layer + ReLU + MaxPool
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten the tensor to (batch_size, 16*5*5)
        x = x.view(-1, 16 * 5 * 5)

        # Apply first fully connected layer + ReLU
        x = F.relu(self.fc1(x))

        # Apply second fully connected layer + ReLU
        x = F.relu(self.fc2(x))

        # Apply final output layer (logits)
        x = self.fc3(x)

        return x


In [15]:
model=CNN()
print(model)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Define loss function and optimizer

In [16]:
import torch.optim as optim


In [17]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)


Training Loop

In [19]:
# Function to train a PyTorch model
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    # Set the model to training mode (important for layers like Dropout or BatchNorm)
    model.train()

    # Loop over the number of epochs
    for epoch in range(epochs):
        running_loss = 0.0  # To accumulate the loss for the epoch

        # Iterate over the training data loader
        for images, labels in train_loader:
            # Zero out gradients from the previous step
            optimizer.zero_grad()

            # Forward pass: get model predictions
            outputs = model(images)

            # Compute the loss between predictions and true labels
            loss = criterion(outputs, labels)

            # Backward pass: compute gradients
            loss.backward()

            # Optimizer step: update model parameters
            optimizer.step()

            # Accumulate the loss
            running_loss += loss.item()

        # Print average loss for the epoch
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}')


# Example call to the function
train_model(model, train_loader, criterion, optimizer, epochs=10)


Epoch 1, Loss: 1.5780
Epoch 2, Loss: 1.3383
Epoch 3, Loss: 1.2163
Epoch 4, Loss: 1.1376
Epoch 5, Loss: 1.0738
Epoch 6, Loss: 1.0206
Epoch 7, Loss: 0.9785
Epoch 8, Loss: 0.9313
Epoch 9, Loss: 0.8960
Epoch 10, Loss: 0.8587


Evaluation Loop

In [21]:
# Function to evaluate the trained model on the test set
def evaluate_model(model, test_loader):
    # Set the model to evaluation mode (deactivates Dropout, etc.)
    model.eval()

    correct = 0  # Counter for correct predictions
    total = 0    # Counter for total samples

    # Disable gradient calculation (saves memory and computation)
    with torch.no_grad():
        # Iterate over the test dataset
        for images, labels in test_loader:
            # Forward pass: get model predictions
            outputs = model(images)

            # Get predicted class (the one with highest probability)
            _, predicted = torch.max(outputs, 1)

            # Update total and correct counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Compute accuracy
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')


# Example usage of the function
evaluate_model(model, test_loader)


Accuracy on the test set: 63.32%
